Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization, Lambda
from keras.models import model_from_json, Model
import keras.backend as K
from keras import regularizers
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import gen_classifier_dataset

import json
import time

import numpy as np
import sklearn

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 5
BINS_EDGE = np.load("./s1p10_data/s1p10_bins_edge.npy")
NUM_CLASSES = len(BINS_EDGE) - 1  

with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    CLASSES_WEIGHT = json.load(fp)

# 1. Model Acrchitect

1 Image -> ResNet-8 -> Dense regressor -> 5 steering angles

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [3]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
# for l in encoder.layers:
#     l.trainable = False
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

## 1.3 Create Decoder
Decoder is a seri of Dense layers with a non-activate Dense layer at the end to perform regression.

## 1.4 Create Model

In [4]:
reshapor = Reshape((1, -1))

def s1p10_model(input_shape):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Dropout(0.5)(X_feature)
    
    # apply classifier body
    X_body = Dense(800, activation='relu')(X_feature)
    
    # apply classifier head
    y = []
    for i in range(NUM_LABELS): 
        out = Dense(NUM_CLASSES, activation='softmax', name="head_%d" % i)(X_body)
        y.append(out)
    
#     # apply classifier head
#     class_neck = []
#     for i in range(NUM_LABELS):
#         X = Dense(300, activation='relu', name="neck_%d" % i)(X_body)
#         X = reshapor(X)
#         class_neck.append(X)
    
#     # apply LSTM
#     X = keras.layers.concatenate(class_neck, axis=1)  # to get shape (None, 5, 300)
#     X = LSTM(128, return_sequences=True, dropout=0.5)(X)
    
#     # apply classifier head
#     y = []
#     for i in range(NUM_LABELS):
#         X_neck = Lambda(lambda X : X[:, i, :])(X)
#         X = Dropout(0.5)(X)
#         out = Dense(NUM_CLASSES, activation="softmax", name="head_%d" % i)(X_neck)
#         y.append(out)
    
    model = Model(inputs=[X_in], outputs=y)
    return model
        

In [5]:
model = s1p10_model(IMAGE_SHAPE)

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 6272)         0           resnet8[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 800)          5018400     dropout_1[0][0]             

In [6]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['acc'])

# Training

In [7]:
gen_param = {'num_classes': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': BINS_EDGE, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': 10, 
             'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'flip_prob': 0.5}

# X_train, y_train = gen_classifier_dataset("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
X_train = np.load('./s1p10_data/CH2_training_X.npy')
y_train_arr = np.load('./s1p10_data/CH2_training_y.npy')  # shape (5, len_dataset, num_classes)
y_train = [y_train_arr[i, :, :]for i in range(NUM_LABELS)]

# X_val, y_val = gen_classifier_dataset("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)
X_val = np.load('./s1p10_data/CH2_validation_X.npy')
y_val_arr = np.load('./s1p10_data/CH2_validation_y.npy')
y_val = [y_val_arr[i, :, :]for i in range(NUM_LABELS)]

In [8]:
batch_size = 200

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

# eva_metric = EVAMetrics()

model.fit(x=X_train,
          y=y_train,
          epochs=40,
          validation_data=(X_val, y_val),
          class_weight=CLASSES_WEIGHT,
          initial_epoch=0,
          shuffle=True,
          batch_size=batch_size,
          callbacks=[tb_callback])

# eva_data = eva_metric.get_data()

Instructions for updating:
Use tf.cast instead.
Train on 18232 samples, validate on 2026 samples
Epoch 1/80
18232/18232 [==============================] - 43s 2ms/step - loss: 19.0990 - head_0_loss: 3.7594 - head_1_loss: 3.8747 - head_2_loss: 3.7757 - head_3_loss: 3.9025 - head_4_loss: 3.7866 - head_0_acc: 0.1966 - head_1_acc: 0.1920 - head_2_acc: 0.1870 - head_3_acc: 0.1849 - head_4_acc: 0.1880 - val_loss: 14.0181 - val_head_0_loss: 2.8251 - val_head_1_loss: 2.8192 - val_head_2_loss: 2.7942 - val_head_3_loss: 2.8004 - val_head_4_loss: 2.7793 - val_head_0_acc: 0.2310 - val_head_1_acc: 0.2310 - val_head_2_acc: 0.2349 - val_head_3_acc: 0.2374 - val_head_4_acc: 0.2399
Epoch 2/80
18232/18232 [==============================] - 40s 2ms/step - loss: 13.5255 - head_0_loss: 2.7272 - head_1_loss: 2.7038 - head_2_loss: 2.6891 - head_3_loss: 2.7067 - head_4_loss: 2.6987 - head_0_acc: 0.2391 - head_1_acc: 0.2402 - head_2_acc: 0.2434 - head_3_acc: 0.2383 - head_4_acc: 0.2396 - val_loss: 12.1822 - va

Epoch 15/80
18232/18232 [==============================] - 40s 2ms/step - loss: 5.7187 - head_0_loss: 1.2249 - head_1_loss: 1.1459 - head_2_loss: 1.0951 - head_3_loss: 1.1035 - head_4_loss: 1.1493 - head_0_acc: 0.5981 - head_1_acc: 0.6191 - head_2_acc: 0.6311 - head_3_acc: 0.6270 - head_4_acc: 0.6136 - val_loss: 5.9882 - val_head_0_loss: 1.3069 - val_head_1_loss: 1.2217 - val_head_2_loss: 1.1547 - val_head_3_loss: 1.1555 - val_head_4_loss: 1.1494 - val_head_0_acc: 0.6037 - val_head_1_acc: 0.6214 - val_head_2_acc: 0.6333 - val_head_3_acc: 0.6500 - val_head_4_acc: 0.6283
Epoch 16/80
18232/18232 [==============================] - 40s 2ms/step - loss: 5.3618 - head_0_loss: 1.1495 - head_1_loss: 1.0748 - head_2_loss: 1.0179 - head_3_loss: 1.0402 - head_4_loss: 1.0794 - head_0_acc: 0.6231 - head_1_acc: 0.6363 - head_2_acc: 0.6577 - head_3_acc: 0.6439 - head_4_acc: 0.6362 - val_loss: 5.7861 - val_head_0_loss: 1.2634 - val_head_1_loss: 1.1888 - val_head_2_loss: 1.1288 - val_head_3_loss: 1.1073

18232/18232 [==============================] - 40s 2ms/step - loss: 2.9648 - head_0_loss: 0.6454 - head_1_loss: 0.5931 - head_2_loss: 0.5544 - head_3_loss: 0.5676 - head_4_loss: 0.6044 - head_0_acc: 0.7809 - head_1_acc: 0.7919 - head_2_acc: 0.8055 - head_3_acc: 0.8021 - head_4_acc: 0.7930 - val_loss: 4.9192 - val_head_0_loss: 1.0851 - val_head_1_loss: 0.9806 - val_head_2_loss: 0.9618 - val_head_3_loss: 0.9423 - val_head_4_loss: 0.9494 - val_head_0_acc: 0.7108 - val_head_1_acc: 0.7206 - val_head_2_acc: 0.7226 - val_head_3_acc: 0.7359 - val_head_4_acc: 0.7256
Epoch 30/80
18232/18232 [==============================] - 40s 2ms/step - loss: 2.8651 - head_0_loss: 0.6264 - head_1_loss: 0.5618 - head_2_loss: 0.5396 - head_3_loss: 0.5511 - head_4_loss: 0.5862 - head_0_acc: 0.7882 - head_1_acc: 0.8071 - head_2_acc: 0.8137 - head_3_acc: 0.8058 - head_4_acc: 0.7949 - val_loss: 5.0137 - val_head_0_loss: 1.1006 - val_head_1_loss: 1.0013 - val_head_2_loss: 0.9823 - val_head_3_loss: 0.9725 - val_head_

KeyboardInterrupt: 

# Save weights

In [9]:
# serialize model to JSON
model_json = model.to_json()
with open(log_dir + "/s1p10_model_%s.json" % time_str, "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(log_dir + "/s1p10_model_%s.h5" % time_str)
print("Saved model to disk")

Saved model to disk


In [ ]:
# regress_json = regressor.to_json()
# with open("./s1p10_model/s1p10_noLSTM_regressor_%s.json" % time_str, "w") as json_file:
#     json_file.write(regress_json)

# save classifier
# regressor.save_weights(log_dir + "/regressor_%s.h5" % time_str)

In [10]:
time_str

'2019_05_30_21_41'

In [ ]:
# # #TODO: save X_train, y_train
# # np.save('./s1p10_data/CH2_training_X.npy', X_train)
# # np.save('./s1p10_data/CH2_training_y.npy', y_train)

# np.save('./s1p10_data/CH2_validation_X.npy', X_val)
# np.save('./s1p10_data/CH2_validation_y.npy', y_val)